In [1]:
import binascii
import hashlib
import math
import struct

import bencodepy
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes


def uint32(data, offset):
    """Reads a 32-bit unsigned integer (little-endian) from the data."""
    return struct.unpack_from("<I", data, offset)[0]

class HAXFile:
    def __init__(self, file_data, keys):
        # Validate header length
        expected_header_length = uint32(file_data, 8)
        if len(file_data) < expected_header_length:
            raise ValueError("Incomplete HAX header")

        # File properties
        self.file_length = uint32(file_data, 4)
        self.header_length = expected_header_length
        self.extra_length = uint32(file_data, 12)

        # Decode metadata (e.g., using Bencode)
        metadata_raw = file_data[16:self.header_length]
        self.meta = bencodepy.decode(metadata_raw)

        # Base key
        self.base_key = self.meta[b"baseKey"]  # Raw base key

        # Parse segments
        self.segments = []
        for i in range(self.meta[b"segmentCount"]):
            offset = uint32(self.meta[b"segments"], i * 8)
            pts = uint32(self.meta[b"segments"], i * 8 + 4)
            self.segments.append({"off": offset, "pts": pts})

        # Add the last segment (end of the file)
        self.segments.append({
            "off": self.file_length,
            "pts": self.meta[b"durationMs"]
        })

        # Load decryption keys - convert hex strings to bytes
        self.keys = {int(k): binascii.unhexlify(v) for k, v in keys.items()}

    def segment_bounds(self, segment_index):
        """Returns the byte range for a given segment."""
        if segment_index < 0 or segment_index >= len(self.segments) - 1:
            return None
        return (self.segments[segment_index]["off"], self.segments[segment_index + 1]["off"])

    def derive_key(self, index):
        """Derives a cryptographic key for a given index."""
        r = math.ceil(math.log2(index)) if index > 0 else 0
        derived_key = None
        found_i = 0

        # Find the first applicable key by checking each bit position
        for i in range(r + 1):
            key_index = index >> (r - i)  # Shift right to get the key index
            if key_index in self.keys:
                derived_key = self.keys[key_index]
                found_i = i
                break

        if derived_key is None:
            # If no key is found, try using the base key
            if 1 in self.keys:  # Often the base key is stored with index 1
                derived_key = self.keys[1]
                found_i = 0
            else:
                raise ValueError("No applicable key available")

        # Iteratively derive the key using the remaining bits
        for i in range(found_i + 1, r + 1):
            shift_byte = bytes([index >> (r - i) & 0xFF])
            derived_key = hashlib.sha256(derived_key + shift_byte).digest()

        return derived_key

    def decode(self, segment_index, raw_data):
        """Decrypts a segment using its index and raw data."""
        # Compute h as in the JavaScript code
        h = 1 + (1 << (math.ceil(math.log2(len(self.segments))) + 1))

        # Derive the decryption key
        derived_key = self.derive_key(h + segment_index)

        # Pad the data to a multiple of 16 bytes if necessary
        padding_length = (16 - len(raw_data) % 16) % 16
        padded_data = raw_data + bytes([padding_length] * padding_length)

        # Initialize decryption with AES in CBC mode with zero IV
        iv = bytes(16)  # 16 bytes of zeros
        cipher = Cipher(algorithms.AES(derived_key), modes.CBC(iv), backend=default_backend())
        decryptor = cipher.decryptor()

        # Decrypt the data
        decrypted_data = decryptor.update(padded_data) + decryptor.finalize()

        # Remove padding
        if padding_length > 0:
            decrypted_data = decrypted_data[:-padding_length]

        return decrypted_data

In [11]:
listen_dict = {
    "url": "https://cdn.hotaudio.net/a/hm6aq9rrzwtt2drebe64hmjf00.hax",
    "length15s": 245797,
    "keys": {
        "32": "2c90525bd6fdc786146e0d239ee1fa73fe19d34b14e251adbd17ed3459b39c91",
        "33": "c1e3916a2fd7ed4912699e7a9055553233d6218ace9ee24b3b3947ee66e2aed1",
        "34": "1c3d68352776b05c29575f3871d2993c45b129c0c83a9a1ff76b0b00f6dca78d",
        "35": "eb35eab8bdf04d53ecaed302fd533c5d95494f24f004a73dbe1bf8b46f0a90c6",
        "36": "7d131390c0048c51608628d1d2fbe7faf93acb28f0c85d0017b82c4f23a44d00",
        "37": "a0e3c6e98423229cf95f73cbf82a48d8bb5533917a39723801d716e6de334559",
        "38": "a8bd5c9544ef675f0948b3e1ed6475375d107d645cf4855ad6792d8c6e1856b0",
        "39": "c08359e3b435a53813d1d622f47620288b8e310cbbd83c39d4e5738109cee7e8",
        "40": "3fa3b14d7ffec9341ac34b54fb136d83bf68bf49060802cc74c53b835aec6a12",
        "41": "8bf9097b1f748fb5413e8185b47d2aa27383ebe39ad746e9b50588fc656b0f04",
        "42": "5aaf87dfd6c3a57ab856acbbed8f95f314739b61fe193aa549ae730bc8958304",
        "43": "1f5f1869a4d260cafab5d497d3ae940e0c09dda4a9493bf0e6cda8cbdf9de19a",
        "44": "d6c00ead34bda427964d19cfa31a334b81c1deed7904682ba891eb8a90621097",
        "45": "e6447e63434987ffbcc9ab89e8408f5aaeaf8f09c59884a5191a24cfe7db43cd",
        "23": "4ed88aedba481a7b496b5c360318e1da0afec20627ccea5e536e17f8b1bdb275"
    }
}

In [5]:
with open("hm6aq9rrzwtt2drebe64hmjf00.hax", "rb") as f:
    file_data = f.read()

hax_file = HAXFile(file_data, listen_dict["keys"])

# Check segment bounds
for i in range(len(hax_file.segments) - 1):
    print(f"Segment {i}: {hax_file.segment_bounds(i)}")


Segment 0: (20139, 32433)
Segment 1: (32433, 46157)
Segment 2: (46157, 59739)
Segment 3: (59739, 74545)
Segment 4: (74545, 88763)
Segment 5: (88763, 103408)
Segment 6: (103408, 119049)
Segment 7: (119049, 133025)
Segment 8: (133025, 147333)
Segment 9: (147333, 164271)
Segment 10: (164271, 178660)
Segment 11: (178660, 193934)
Segment 12: (193934, 210126)
Segment 13: (210126, 228186)
Segment 14: (228186, 245797)
Segment 15: (245797, 261949)
Segment 16: (261949, 279606)
Segment 17: (279606, 298275)
Segment 18: (298275, 311834)
Segment 19: (311834, 327803)
Segment 20: (327803, 341231)
Segment 21: (341231, 355947)
Segment 22: (355947, 375297)
Segment 23: (375297, 393242)
Segment 24: (393242, 408973)
Segment 25: (408973, 428574)
Segment 26: (428574, 447053)
Segment 27: (447053, 464055)
Segment 28: (464055, 479016)
Segment 29: (479016, 495434)
Segment 30: (495434, 513508)
Segment 31: (513508, 529774)
Segment 32: (529774, 545439)
Segment 33: (545439, 561603)
Segment 34: (561603, 577402)
Segmen

In [4]:
with open("hm6aq9rrzwtt2drebe64hmjf00.hax", "rb") as f:
    file_data = f.read()

hax_file = HAXFile(file_data, listen_dict["keys"])

# Decode a specific segment (replace 0 with the actual segment index)
segment_bounds = hax_file.segment_bounds(0)
raw_data = file_data[segment_bounds[0]:segment_bounds[1]]
decrypted_data = hax_file.decode(0, raw_data)

print(decrypted_data)


b'\xddU\xfe]g\x918H3\xd6\x8f}^W\xbd\xda\x08e\x98\xf9\xc8\xdbApE\xb8V)g\x01)\x8c\xf0\xf7\xe8\xad\xf1v\x8a\xcdky*\xb8\xd3\x8b0o`\xd6Z/\xb1\xde \xca\xf9b\xb5\x05t\x1a\xb0\xd8\xb1\x85\x8eg\xac\x80}VH\x97P\xd3_|BV<\x94c\xbd\x8e\xd4\xbf=\xf4N\xa9\xd3\x08\xb5\xf0\x04c\xb8\xbe\xb1D\xa0#\xb2Pw\x94\xf1k\x05\xa5\xba\x84s,\x96;~7(\xb5\x16\xc6\x1c\xa6\x88]\xe5d\x86\xc87\x83\xd7\x16ew\xbe\x0ef\xae\xa5\x97.w\xb8F\\\xc1\xde\x80\'j&\x1a\x85\xab\x17\x08\xb5\\\x96x\x05@\x13\x17E\xc3q\x92\x86\xf8\xb4\x10Y9\x98t\xaeE\xed\xbb\xe8\x1b\x8b=}\x13\xbc\xc7\xd6\xe6\x14\xd2\x1a\xb0\xfb\x17\xee\xac\xc3\x9c5\xf6\x91\x8c\xdf\x9f\x9c\xa2\xa3\x93\x93zF\xcb\xcc+*v\x0e\xe8\xd7\xd34\xebZI\x93\xf2\x10\xc8\xa2V\xf2\x8dlX\x973 \xc5qNB\x1d\x13\x0c\xd18\xf7N=\x19\xeb\x98T\xbc\xf0\xe3:\xe4\r\xd5\xd2\xa4\xdb\x96\xb7\xed\xeea\xf8@q\xb7M\x97\x02\xbaM\x8c{K\xa0\xe4\xa5f\xdc\x1bh\xe7\x1b^C\xd7\xdd\x85\x0b\xd2\xdd\xd7\x8ae\xbb\xc7\xd0\xe0\x88\ts\xc2\xa7\x88\xd0\t\x19\x87\xc5\xf0>\xd2\xf69\xa5I\xa6\xa8z\xceB\xfc\xf5\xae\x11Eg\xdc\x87\

In [12]:
with open("hm6aq9rrzwtt2drebe64hmjf00.hax", "rb") as f:
    file_data = f.read()

hax_file = HAXFile(file_data, listen_dict["keys"])

# Create output AAC file
with open("output.aac", "wb") as out_f:
    # Decode all segments
    for i in range(len(hax_file.segments) - 1):  # -1 because last segment is just end marker
        segment_bounds = hax_file.segment_bounds(i)
        if segment_bounds:
            raw_data = file_data[segment_bounds[0]:segment_bounds[1]]
            try:
                decrypted_data = hax_file.decode(i, raw_data)
                out_f.write(decrypted_data)
                print(f"Decoded segment {i}/{len(hax_file.segments)-2}")
            except Exception as e:
                print(f"Error decoding segment {i}: {e}")
                continue

print("Decoding complete. Output saved to output.aac")

Decoded segment 0/1879
Decoded segment 1/1879
Decoded segment 2/1879
Decoded segment 3/1879
Decoded segment 4/1879
Decoded segment 5/1879
Decoded segment 6/1879
Decoded segment 7/1879
Decoded segment 8/1879
Decoded segment 9/1879
Decoded segment 10/1879
Decoded segment 11/1879
Decoded segment 12/1879
Decoded segment 13/1879
Decoded segment 14/1879
Decoded segment 15/1879
Decoded segment 16/1879
Decoded segment 17/1879
Decoded segment 18/1879
Decoded segment 19/1879
Decoded segment 20/1879
Decoded segment 21/1879
Decoded segment 22/1879
Decoded segment 23/1879
Decoded segment 24/1879
Decoded segment 25/1879
Decoded segment 26/1879
Decoded segment 27/1879
Decoded segment 28/1879
Decoded segment 29/1879
Decoded segment 30/1879
Decoded segment 31/1879
Decoded segment 32/1879
Decoded segment 33/1879
Decoded segment 34/1879
Decoded segment 35/1879
Decoded segment 36/1879
Decoded segment 37/1879
Decoded segment 38/1879
Decoded segment 39/1879
Decoded segment 40/1879
Decoded segment 41/1879
De